In [ ]:
"""
Practicing an learning LDA topic modeling by following Susan Li's 
"Topic Modeling and Latent Dirichlet Allocation (LDA) in Python" 
 at https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
"""

In [27]:
# lda practice

In [28]:
import pandas as pd

In [29]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [30]:
print(len(documents))
print(documents[:10])

1186018
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4
5                  ambitious olsson wins triple jump      5
6         antic delighted with record breaking barca      6
7  aussie qualifier stosur wastes four memphis match      7
8       aust addresses un security council over iraq      8
9         australia is locked into war timetable opp      9


In [31]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
stemmer = PorterStemmer()


In [32]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Mitchellmorrison/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))


def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [34]:
doc_sample = documents[documents['index'] == 4010].values[0][0]

print('original doucment')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenize and lemmatized document: ')
print(preprocess(doc_sample))

original doucment
['greens', 'divided', 'over', 'preference', 'distribution']


 tokenize and lemmatized document: 
['green', 'divid', 'prefer', 'distribut']


In [35]:
processed_docs = []
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [36]:
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [37]:
# remove any words w less than 15 instances overall, best 100000 words
dictionary.filter_extremes(no_below=15, no_above=.5, keep_n=100000)

In [38]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4010]

[(234, 1), (2283, 1), (2743, 1), (3804, 1)]

In [46]:
# frequency of each word in headline for specific headline

bow_doc_4010 = bow_corpus[4010]

for i in range(len(bow_doc_4010)):
    print("Word {} (\"{}\") appears {} times.".format
          (bow_doc_4010[i][0], 
           dictionary[bow_doc_4010[i][0]], 
           bow_doc_4010[i][1]))

Word 234 ("green") appears 1 times.
Word 2283 ("divid") appears 1 times.
Word 2743 ("prefer") appears 1 times.
Word 3804 ("distribut") appears 1 times.


In [47]:
#get tf/idf ratios for each word

from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.5860586362613176), (1, 0.3854657616151764), (2, 0.5006618583937537), (3, 0.5072367544211179)]


In [48]:
# lda using bag of words

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [49]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"market" + 0.022*"school" + 0.016*"investig" + 0.015*"interview" + 0.014*"street" + 0.014*"fall" + 0.013*"student" + 0.012*"scott" + 0.011*"share" + 0.010*"week"
Topic: 1 
Words: 0.020*"donald" + 0.018*"plan" + 0.018*"canberra" + 0.013*"council" + 0.011*"water" + 0.011*"feder" + 0.011*"drum" + 0.010*"industri" + 0.010*"concern" + 0.010*"park"
Topic: 2 
Words: 0.042*"australia" + 0.037*"trump" + 0.024*"world" + 0.016*"open" + 0.015*"tasmania" + 0.014*"final" + 0.014*"australian" + 0.012*"win" + 0.011*"break" + 0.010*"record"
Topic: 3 
Words: 0.029*"queensland" + 0.021*"help" + 0.017*"miss" + 0.016*"royal" + 0.015*"tasmanian" + 0.014*"bank" + 0.013*"victoria" + 0.013*"report" + 0.013*"commiss" + 0.011*"drought"
Topic: 4 
Words: 0.024*"news" + 0.022*"women" + 0.021*"live" + 0.020*"warn" + 0.020*"coast" + 0.019*"health" + 0.016*"rural" + 0.015*"busi" + 0.015*"countri" + 0.013*"gold"
Topic: 5 
Words: 0.053*"polic" + 0.049*"say" + 0.024*"kill" + 0.023*"attack" + 0.021*

In [52]:
# lda using tf/idf

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} \nWord: {}".format(idx, topic))

Topic: 0 
Word: 0.007*"weather" + 0.006*"water" + 0.006*"govern" + 0.006*"bushfir" + 0.006*"andrew" + 0.005*"hobart" + 0.005*"council" + 0.005*"action" + 0.005*"concern" + 0.005*"plan"
Topic: 1 
Word: 0.019*"polic" + 0.018*"charg" + 0.016*"murder" + 0.012*"court" + 0.012*"woman" + 0.011*"death" + 0.010*"jail" + 0.010*"alleg" + 0.009*"arrest" + 0.009*"sentenc"
Topic: 2 
Word: 0.022*"trump" + 0.013*"interview" + 0.008*"final" + 0.007*"leagu" + 0.007*"world" + 0.007*"australia" + 0.006*"michael" + 0.006*"scott" + 0.006*"morrison" + 0.006*"grandstand"
Topic: 3 
Word: 0.015*"elect" + 0.011*"stori" + 0.008*"wall" + 0.008*"liber" + 0.008*"street" + 0.007*"labor" + 0.007*"parti" + 0.006*"polit" + 0.006*"retir" + 0.006*"octob"
Topic: 4 
Word: 0.013*"market" + 0.009*"price" + 0.007*"share" + 0.007*"australian" + 0.007*"rise" + 0.007*"cattl" + 0.007*"rat" + 0.006*"live" + 0.006*"peter" + 0.006*"financ"
Topic: 5 
Word: 0.026*"news" + 0.018*"rural" + 0.016*"drum" + 0.010*"tuesday" + 0.010*"friday" 